In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image

/home/viktor/programs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
IMG_SIZE = (40, 40)
IMG_SHAPE = (40, 40, 3)
NOF_CLASSES = 43

TRAIN_SET_PATH = '/home/viktor/workspace/00_datasets/GTSRB_40x40/training_set_40x40'
TRAIN_SET_SIZE = 39209
BATCH_SIZE = 128
TRAIN_STEPS = TRAIN_SET_SIZE // BATCH_SIZE

VALID_SET_PATH = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40'
VALID_SET_SIZE = 12630
VALID_BATCH_SIZE = 64
VALID_STEPS = VALID_SET_SIZE // VALID_BATCH_SIZE

In [3]:
# Flow training images in batches of 128 using train_datagen generator
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        TRAIN_SET_PATH,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='sparse')

valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        VALID_SET_PATH,
        target_size=IMG_SIZE,
        batch_size=VALID_BATCH_SIZE,
        class_mode='sparse')

Found 39209 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=IMG_SHAPE),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(input_shape=IMG_SHAPE),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(NOF_CLASSES, activation='softmax')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
__________

In [5]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [6]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=TRAIN_STEPS,  
      epochs=15,
      verbose=2,
      validation_data = valid_generator,
      validation_steps=VALID_STEPS)

Epoch 1/15
 - 14s - loss: 1.5723 - acc: 0.5646 - val_loss: 0.8091 - val_acc: 0.7859
Epoch 2/15
 - 6s - loss: 0.2951 - acc: 0.9166 - val_loss: 0.4347 - val_acc: 0.9020
Epoch 3/15
 - 6s - loss: 0.1075 - acc: 0.9703 - val_loss: 0.4704 - val_acc: 0.8972
Epoch 4/15
 - 6s - loss: 0.0593 - acc: 0.9836 - val_loss: 0.3391 - val_acc: 0.9308
Epoch 5/15
 - 6s - loss: 0.0355 - acc: 0.9905 - val_loss: 0.3424 - val_acc: 0.9338
Epoch 6/15
 - 6s - loss: 0.0239 - acc: 0.9935 - val_loss: 0.3344 - val_acc: 0.9366
Epoch 7/15
 - 6s - loss: 0.0189 - acc: 0.9947 - val_loss: 0.3146 - val_acc: 0.9479
Epoch 8/15
 - 6s - loss: 0.0149 - acc: 0.9958 - val_loss: 0.3492 - val_acc: 0.9442
Epoch 9/15
 - 6s - loss: 0.0109 - acc: 0.9972 - val_loss: 0.3347 - val_acc: 0.9475
Epoch 10/15
 - 6s - loss: 0.0098 - acc: 0.9971 - val_loss: 0.3760 - val_acc: 0.9438
Epoch 11/15
 - 6s - loss: 0.0098 - acc: 0.9977 - val_loss: 0.4346 - val_acc: 0.9400
Epoch 12/15
 - 6s - loss: 0.0083 - acc: 0.9979 - val_loss: 0.3602 - val_acc: 0.9507


In [7]:
img = image.load_img('/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/00017/00031.ppm',
                     target_size=IMG_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(np.where(classes[0])[0])

[17]


In [8]:
# Save entire model to a HDF5 file
model.save('gtsrb_01_model.h5')

In [35]:
from shutil import move
df = pd.read_csv('/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/GT-final_test.csv', delimiter=';')
for index, row in df.iterrows():
    src_dir = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40'
    src_file = src_dir + '/' + row['Filename']
    dst_dir = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/' + f'{row["ClassId"]:05d}'
    dst_file = dst_dir + '/' + row['Filename']
    
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    move(src_file, dst_file)